# <center>TP 4</center>


- Initialisation de l'environnement et des structures de donnees

In [2]:
import gymnasium as gym
import numpy as np

#initialisation de l'environnement

env=gym.make("Taxi-v3")

#Nombre des etat et des actions
state_size = env.observation_space.n
action_size = env.action_space.n

#Initialize policy and value tables
policy_table=np.ones((state_size,action_size))/action_size
value_table =np.zeros(state_size)

#Display first lines of tables
print("Policy Table Sample: \n",policy_table[:5])
print("Value Table Sample: \n",value_table[:5])

Policy Table Sample: 
 [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
Value Table Sample: 
 [0. 0. 0. 0. 0.]


- Exploration et collecte d’épisodes

In [3]:
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0  
    step_count = 0  

    print(f"\nÉpisode {episode + 1}:")
    
    while not done:
        action = env.action_space.sample()  # Sélectionner une action aléatoire
        next_state, reward, done, _, _ = env.step(action)  # Appliquer l’action
        total_reward += reward  # Ajouter la récompense à la somme
        step_count += 1

        print(f"  Action: {action}, Récompense: {reward}, État suivant: {next_state}")

    print(f"  → Durée de l’épisode: {step_count} étapes, Récompense totale: {total_reward}")

env.close()



Épisode 1:
  Action: 3, Récompense: -1, État suivant: 4
  Action: 3, Récompense: -1, État suivant: 4
  Action: 1, Récompense: -1, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 1, Récompense: -1, État suivant: 4
  Action: 2, Récompense: -1, État suivant: 24
  Action: 5, Récompense: -10, État suivant: 24
  Action: 3, Récompense: -1, État suivant: 4
  Action: 1, Récompense: -1, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 3, Récompense: -1, État suivant: 4
  Action: 5, Récompense: -10, État suivant: 4
  Action: 4, Récompense: -10, État suivant: 4
  Action: 0, Récompense: -1, État suivant: 104
  Action: 4, Récompense: -10, État suivant: 104
  Action: 5, Récompense: -10, État suivant: 104
  Action: 3, Récompense: -1, État suivant: 104
  Action: 3, Récompense: -1, État suivant: 104
  Action: 2, Récompen

- Mise a jour de la politique avec PPO

In [4]:
# Hyperparamètres
gamma = 0.99  
lr_policy = 0.1  
clip_epsilon = 0.2  


# Fonction pour calculer les récompenses cumulées (discounted rewards)
def compute_discounted_rewards(rewards, gamma):
    discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
    cumulative = 0
    for t in reversed(range(len(rewards))):
        cumulative = rewards[t] + gamma * cumulative
        discounted_rewards[t] = cumulative
    return discounted_rewards

# Fonction de mise à jour PPO
def update_policy(episode_states, episode_actions, episode_rewards):
    global value_table, policy_table

    # Calcul des récompenses cumulées
    discounted_rewards = compute_discounted_rewards(episode_rewards, gamma)

    # Calcul de l’avantage A_t = R_t - V(s_t)
    advantages = discounted_rewards - value_table[episode_states]

    # Mise à jour de la politique avec PPO (avec clipping)
    for i, state in enumerate(episode_states):
        action = episode_actions[i]
    advantage = advantages[i]

    old_prob = policy_table[state, action]  # Ancienne probabilité d’action
    new_prob = old_prob + lr_policy * advantage  # Mise à jour
    new_prob = np.clip(new_prob, 0, 1)  # Assurer que la probabilité reste entre 0 et 1

    # Mise à jour des probabilités de la politique
    policy_table[state, action] = new_prob
    policy_table[state, :] = policy_table[state, :] / np.sum(policy_table[state, :])  # Normalisation


    # Mise à jour de la fonction de valeur V(s) avec une learning rate
    for i, state in enumerate(episode_states):
        value_table[state] += lr_policy * advantages[i]

# Exécution de plusieurs épisodes pour mettre à jour la politique
num_episodes = 100  # Nombre d'épisodes pour entraîner PPO

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    episode_states = []
    episode_actions = []
    episode_rewards = []

    while not done:
        # Sélection d’une action basée sur la politique actuelle
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, done, _, _ = env.step(action)

        # Stocker les données pour la mise à jour PPO
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)

        state = next_state

    # Mise à jour de la politique après l’épisode
    update_policy(episode_states, episode_actions, episode_rewards)

    if episode % 10 == 0:
        print(f"Épisode {episode}: Politique mise à jour")
        

env.close()


Épisode 0: Politique mise à jour
Épisode 10: Politique mise à jour
Épisode 20: Politique mise à jour
Épisode 30: Politique mise à jour
Épisode 40: Politique mise à jour
Épisode 50: Politique mise à jour
Épisode 60: Politique mise à jour
Épisode 70: Politique mise à jour
Épisode 80: Politique mise à jour
Épisode 90: Politique mise à jour


- Évaluation de l'agent après entraînement

In [ ]:
num_eval_episodes = 20
total_rewards = []
success_count = 0

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0
    
    while not done and steps:
        action = np.argmax(policy_table[state])
        
        next_state, reward, done, truncated, _ = env.step(action)
        done = done or truncated
        
        total_reward += reward
        state = next_state
        steps += 1
    
    total_rewards.append(total_reward)
    if reward > 0:
        success_count += 1

avg_reward = np.mean(total_rewards)
success_rate = (success_count / num_eval_episodes) * 100

print("\nRésultats après entraînement :")
print(f"- Récompense moyenne : {avg_reward:.2f}")
print(f"- Taux de réussite : {success_rate:.2f}%")